In [ ]:
#@title 1. 掛載 Google Drive { display-mode: "form" }

# ==============================================================================
# 目的：
# 建立 Colab 環境與 Google Drive 之間的連接，並設定專案資料夾結構。
#
# 執行動作：
# 1. 觸發授權：執行後會要求授權 Colab 存取您的 Google Drive。
# 2. 掛載硬碟：將 Google Drive 掛載到 /content/drive/MyDrive。
# 3. 建立目錄：自動在您的雲端硬碟建立專案根目錄及 data, models, results 子目錄。
#
# 產出：
# - Colab 獲得對 Google Drive 的讀寫權限。
# - 一個標準化的專案資料夾結構。
# ==============================================================================

from google.colab import drive
import os

#@markdown ### **設定 Google Drive 專案目錄**
#@markdown 請輸入您希望在 Google Drive 中使用的專案根目錄名稱。
DRIVE_PROJECT_ROOT = "MyQuantProject" #@param {type:"string"}
# --- 設定區結束 ---

try:
    # 掛載 Google Drive
    print("正在嘗試掛載 Google Drive...")
    drive.mount('/content/drive', force_remount=True)
    gdrive_path = os.path.join('/content/drive/MyDrive', DRIVE_PROJECT_ROOT)
    
    # 將雲端路徑設定為環境變數，供後續儲存格使用
    os.environ['GDRIVE_PROJECT_PATH'] = gdrive_path
    # 同時也將它設定為預設的專案路徑
    os.environ['PROJECT_ROOT_PATH'] = gdrive_path
    
    print(f"✅ Google Drive 掛載成功！專案雲端路徑設定為：{gdrive_path}")
    
    # 檢查並在雲端硬碟上建立所需的核心資料夾
    for folder in ['data', 'models', 'results']:
        folder_path = os.path.join(gdrive_path, folder)
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
            print(f"📁 已在雲端建立資料夾：{folder_path}")
            
except Exception as e:
    # 如果掛載失敗，清除雲端路徑的環境變數，並給予提示
    os.environ['GDRIVE_PROJECT_PATH'] = ""
    print(f"⚠️ 掛載 Google Drive 失敗或未授權。錯誤：{e}")
    print("後續儲存格將使用 Colab 本機儲存空間（成果不會永久保存）。")

In [ ]:
#@title 2. 同步 GitHub 專案程式碼 { display-mode: "form" }

# ==============================================================================
# 目的：
# 從 GitHub 獲取最新版本的 Python 程式碼。
#
# 執行動作：
# 1. 檢查本地狀態：檢查 Colab 是否已存在專案的程式碼資料夾。
# 2. 下載或更新：
#    - 如果不存在：執行 `git clone` 完整下載專案。
#    - 如果已存在：執行 `git pull` 更新至最新版本。
#
# 產出：
# - Colab 的 /content/ 目錄下，擁有一份與 GitHub 同步的最新程式碼。
# ==============================================================================

import os

#@markdown ### **1. 設定 GitHub 儲存庫**
#@markdown 請填寫您的 GitHub 儲存庫網址。
GITHUB_REPO_URL = "https://github.com/hsp1234-web/qlib.git" #@param {type:"string"}

#@markdown ---
#@markdown ### **2. 輸入分支名稱**
#@markdown 請輸入您想要使用的分支（例如 `main` 或 `develop`）。
GIT_BRANCH = "main" #@param {type:"string"}
# --- 設定區結束 ---

# --- 將設定儲存為環境變數，供後續儲存格使用 ---
os.environ['GITHUB_REPO_URL'] = GITHUB_REPO_URL
os.environ['GIT_BRANCH'] = GIT_BRANCH
# ---

# 從網址中提取專案名稱
repo_name = GITHUB_REPO_URL.split('/')[-1].replace('.git', '')
local_repo_path = os.path.join('/content', repo_name)

print(f"目標 GitHub 儲存庫：{GITHUB_REPO_URL}")
print(f"預計本地路徑：{local_repo_path}")

# 檢查本地資料夾是否已存在
if os.path.exists(local_repo_path):
    print(f"專案資料夾已存在，正在切換至 '{GIT_BRANCH}' 分支並更新...")
    os.chdir(local_repo_path)
    # 使用 -q (quiet) 來減少不必要的輸出
    get_ipython().system('git fetch -q origin')
    get_ipython().system(f'git checkout -q {GIT_BRANCH}')
    get_ipython().system(f'git pull -q origin {GIT_BRANCH}')
    os.chdir('/content')
else:
    print(f"專案資料夾不存在，正在從 '{GIT_BRANCH}' 分支下載整個專案...")
    get_ipython().system(f'git clone -q --branch {GIT_BRANCH} {GITHUB_REPO_URL}')

print(f"\n✅ 程式碼同步完成！已切換至 '{GIT_BRANCH}' 分支。")
# 顯示目前資料夾結構，以確認程式碼已成功下載
print("\n--- 目前 /content/ 目錄結構 ---")
get_ipython().system('ls -F /content/')

In [ ]:
#@title 3. 安裝 Python 套件 { display-mode: "form" }

# ==============================================================================
# 目的：
# 確保 Colab 環境安裝了所有執行專案所需的 Python 套件。
#
# 執行動作：
# 1. 讀取設定檔：找到上一步從 GitHub 同步下來的 `requirements.txt` 檔案。
# 2. 安裝套件：使用 `pip install` 指令，自動安裝或更新清單中指定的所有函式庫。
#
# 產出：
# - 一個準備就緒、包含所有依賴項的 Python 執行環境。
# ==============================================================================

import os

# --- 從環境變數讀取設定 ---
GITHUB_REPO_URL = os.environ.get('GITHUB_REPO_URL')
if not GITHUB_REPO_URL:
    raise ValueError("錯誤：找不到 GITHUB_REPO_URL 環境變數。請先執行儲存格 2。")
# ---

# 從網址中動態推斷專案名稱
repo_name = GITHUB_REPO_URL.split('/')[-1].replace('.git', '')
requirements_path = os.path.join('/content', repo_name, 'requirements.txt')

print(f"正在讀取設定檔：{requirements_path}")

if os.path.exists(requirements_path):
    print("找到 requirements.txt，開始安裝/更新環境...")
    # 使用 -q (quiet) 參數來減少不必要的輸出訊息
    get_ipython().system(f'pip install -q -r {requirements_path}')
    print("\n✅ 環境安裝完成！")
else:
    print(f"⚠️ 警告：在 '{requirements_path}' 中找不到 requirements.txt 檔案。")
    print("將跳過此步驟，但後續程式可能會因缺少套件而執行失敗。")

In [ ]:
#@title 4. 執行回測 { display-mode: "form" }

# ==============================================================================
# 目的：
# 執行主要的回測任務。
#
# 執行動作：
# 1. 設定參數：透過下方的互動式表單，設定回測參數。
# 2. 調用腳本：執行後，系統會自動調用 `backtest_runner.py` 腳本並傳入參數。
# 3. 執行流程：腳本會完成資料下載、策略回測、績效分析等所有步驟。
#
# 產出：
# - 輸出區塊會顯示詳細的回測過程日誌及最終的績效報告。
# - 交易紀錄和分析結果會被儲存到 `results` 資料夾中。
# ==============================================================================

import os

#@markdown ### **回測參數設定**
#@markdown 請在下方輸入您想要回測的參數。
TICKER = "^TWII" #@param {type:"string"}
START_DATE = "2020-01-01" #@param {type:"date"}
END_DATE = "2023-01-01" #@param {type:"date"}
STRATEGY = "defaultlong" #@param ["defaultlong", "defaultshort", "defaultall"]
# --- 設定區結束 ---

# --- 執行回測 ---
# 從環境變數中讀取 GitHub repo URL
GITHUB_REPO_URL = os.environ.get('GITHUB_REPO_URL')
if not GITHUB_REPO_URL:
    raise ValueError("錯誤：找不到 GITHUB_REPO_URL 環境變數。請先執行儲存格 2。")

# 從 repo URL 推斷專案名稱
repo_name = GITHUB_REPO_URL.split('/')[-1].replace('.git', '')
script_path = f"/content/{repo_name}/src/backtest_runner.py"

# 檢查腳本是否存在
if not os.path.exists(script_path):
    raise FileNotFoundError(f"錯誤：找不到回測腳本 '{script_path}'。請確認儲存格 2 已成功執行。")

print(f"--- 執行回測：{TICKER} ---")
command = (
    f"python {script_path} "
    f"--ticker \"{TICKER}\" "
    f"--start_date \"{START_DATE}\" "
    f"--end_date \"{END_DATE}\" "
    f"--strategy {STRATEGY}"
)

print(f"執行命令: {command}")
get_ipython().system(command)

print("\n✅ 回測執行完畢！")

In [ ]:
#@title 5. (可選) 同步成果至 Google Drive { display-mode: "form" }

# ==============================================================================
# 目的：
# 將儲存在 Colab 本機的成果（如 `results` 資料夾）同步回 Google Drive 以永久保存。
# 如果您在未掛載 Google Drive 的情況下執行了回測，此步驟特別有用。
#
# 執行動作：
# 1. 檢查路徑：檢查 Google Drive 是否已掛載。
# 2. 掛載（如果需要）：如果未掛載，會自動執行掛載流程。
# 3. 複製檔案：將 Colab 本機的專案資料夾內容完整複製到 Google Drive。
#
# 產出：
# - 您在本機的所有成果，都會被完整地備份一份到 Google Drive 中。
# ==============================================================================

import os
from google.colab import drive
from distutils.dir_util import copy_tree

# --- 從環境變數讀取雲端路徑 ---
gdrive_path = os.environ.get('GDRIVE_PROJECT_PATH')
# 獲取當前設定的（可能是本機的）專案根目錄
project_root = os.environ.get('PROJECT_ROOT_PATH')

# 檢查 project_root 是否存在
if not project_root or not os.path.exists(project_root):
    print("❌ 錯誤：找不到任何本地專案成果。請先執行儲存格 4。")
# 檢查本地路徑是否已與雲端路徑相同
elif gdrive_path and project_root == gdrive_path:
    print("✅ 您的專案成果已經儲存在 Google Drive 中，無需同步。")
else:
    print("🔄 開始將本地成果同步至 Google Drive...")
    
    # 如果雲端路徑不存在 (儲存格 1 未執行或失敗)，則需要先掛載
    if not gdrive_path:
        try:
            print("正在嘗試掛載 Google Drive...")
            drive.mount('/content/drive', force_remount=True)
            
            # 從儲存格 1 的參數中讀取預設的專案名稱
            # 注意：這裡的 "MyQuantProject" 必須與儲存格 1 的預設值一致
            project_name = "MyQuantProject" 
            gdrive_path = os.path.join('/content/drive/MyDrive', project_name)
            os.environ['GDRIVE_PROJECT_PATH'] = gdrive_path
            print(f"✅ Google Drive 掛載成功！雲端路徑設定為：{gdrive_path}")
        except Exception as e:
            print(f"❌ 掛載 Google Drive 失敗。無法同步。錯誤：{e}")
            gdrive_path = None # 將 gdrive_path 設為 None 以跳過後續操作

    # 如果成功獲取到雲端路徑，則開始複製
    if gdrive_path:
        print(f"正在將 '{project_root}' 的內容複製到 '{gdrive_path}'...")
        
        # 確保雲端目標資料夾存在
        os.makedirs(gdrive_path, exist_ok=True)
        
        # 執行複製
        copy_tree(project_root, gdrive_path)
        
        print("✅ 同步完成！所有本地成果已成功複製到 Google Drive。")